# Experimental Code

<i>Version 1.0</i>

## Dependencies

The package, [bitsets](https://bitsets.readthedocs.io/en/stable/), provides a memory-efficient pure-python immutable ordered set data type for working with large numbers of subsets from a predetermined pool of objects.

In [1]:
from bitsets import bitset

Algebra's are defined in JSON format.

In [2]:
import json

We need the path to this project:

In [3]:
import os

In [4]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Algebras use Bitsets to Represent Sets of Relations

In [5]:
# TODO: Don't embed the abbreviation dictionary in code; create a file for it
def abbrev(term_list):
    abbrev_dict = {"Point":"Pt",
                   "ProperInterval":"PInt",
                   "Interval":"Int"}
    return '|'.join([abbrev_dict[term] for term in term_list])

In [6]:
abbrev(["Point", "ProperInterval"])

'Pt|PInt'

In [7]:
abbrev(['Point'])

'Pt'

In [8]:
class Algebra(object):

    def __init__(self, filename):
        """An algebra is created from a JSON file containing the algebra's
        relation and transitivity table definitions.
        """
        with open(filename, 'r') as f:
            self.algebra_dict = json.load(f)

        self.name = self.algebra_dict["Name"]
        self.description = self.algebra_dict["Description"]
        # TODO: For consistency, rename rel_info to rel_dict
        self.rel_info = self.algebra_dict["Relations"]
        # TODO: Maybe rename AlgBitSet to, say, alg_rels (or allrels_bitset, or ...)
        self.AlgBitSet = bitset(self.name, tuple(self.rel_info.keys()))  # A class object
        # TODO: Remove identity, and just use AlgBitSet.supremum
        self.identity = self.AlgBitSet.supremum

        # Setup the transitivity table used by Relation Set multiplication
        self.transitivity_table = dict()
        tabledefs = self.algebra_dict["TransTable"]
        for rel1 in tabledefs:
            self.transitivity_table[rel1] = dict()
            for rel2 in tabledefs[rel1]:
                self.transitivity_table[rel1][rel2] = self.AlgBitSet(tuple(tabledefs[rel1][rel2]))
                
    # Accessors for information about a given relation, rel.
    def rel_name(self, rel):
        return self.rel_info[rel]["Name"]
    def rel_domain(self,rel):
        return self.rel_info[rel]["Domain"]
    def rel_range(self,rel):
        return self.rel_info[rel]["Range"]
    def rel_reflexive(self,rel):
        return self.rel_info[rel]["Reflexive"]
    def rel_symmetric(self,rel):
        return self.rel_info[rel]["Symmetric"]
    def rel_transitive(self, rel):
        return self.rel_info[rel]["Transitive"]
    
    def converse(self, rel_or_relset):
        '''Return the converse of a relation (str) or relation set (bitset).'''
        if isinstance(rel_or_relset, str):
            return self.rel_info[rel_or_relset]["Converse"]
        else:
            return self.AlgBitSet((self.converse(r) for r in rel_or_relset.members()))
        
    def __str__(self):
        """Return a string representation of the Algebra."""
        return f"<{self.name}: {self.description}>"
    
    def relset(self, elements):
        """Return a relation set (bitset) for the given elements."""
        return self.AlgBitSet(elements)
    
    def add(self, relset1, relset2):
        '''Addition for relation sets is equivalent to set intersection.'''
        return relset1.intersection(relset2)
    
    def mult(self, relset1, relset2):
        '''Multiplication is done, element-by-element, on the cross-product
        of the two sets using the algebra's transitivity table, and
        then reducing those results to a single relation set using set
        union.
        '''
        result = self.AlgBitSet.infimum  # the empty relation set
        for r1 in relset1:
            for r2 in relset2:
                result = result.union(self.transitivity_table[r1][r2])
        return result
    
    def check_multiplication_identity(self, verbose=False):
        """Check the validity of the multiplicative identity for every
        combination of singleton relset.  :param verbose: Print out
        the details of each test :return: True or False

        """
        count = 0
        result = True
        rels = self.identity
        for r in rels:
            r_rs = self.relset((r,))
            for s in rels:
                count += 1
                s_rs = self.relset((s,))
                prod1 = self.mult(r_rs, s_rs)
                prod2 = self.converse(self.mult(self.converse(s_rs), self.converse(r_rs)))
                if prod1 != prod2:
                    if verbose:
                        print("FAIL:")
                        print(f"      r    = {r_rs}")
                        print(f"      s    = {s_rs}")
                        print(f"( r *  s)  = {prod1}")
                        print(f"(si * ri)i = {prod2}")
                        print(f"{prod1} != {prod2}")
                    result = False
        if verbose:
            print(f"\n{self.name} -- Multiplication Identity Check:")
        if result:
            if verbose:
                print(f"PASSED . {count} products tested.")
        else:
            if verbose:
                print("FAILED. See FAILURE output above.")
        return result

    def print_info(self):
        print(f"  Algebra Name: {self.name}")
        print(f"   Description: {self.description}")
        # print(f" Equality Rels: {self.equality_relations}")
        print("     Relations:")
        print("{:>25s} {:>25s} {:>10s} {:>10s} {:>10s} {:>8s} {:>12s}".format("NAME (ABBREV)", "CONVERSE (ABBREV)",
                                                                              "REFLEXIVE", "SYMMETRIC", "TRANSITIVE",
                                                                              "DOMAIN", "RANGE"))
        # TODO: Vary spacing between columns based on max word lengths
        for r in self.identity:
            print(f"{self.rel_name(r):>19s} ({r:>3s}) " \
                  f"{self.rel_name(self.converse(r)):>19s} ({self.converse(r):>3s}) " \
                  f"{self.rel_reflexive(r)!s:>8} {self.rel_symmetric(r)!s:>10} {self.rel_transitive(r)!s:>10}" \
                  f"{abbrev(self.rel_domain(r))!s:>11} {abbrev(self.rel_range(r))!s:>13}")
        # TODO: Don't hardcode the legend below; make it depend on an abbreviations file (JSON)
        print("\nDomain & Range Abbreviations:")
        print("   Pt = Point")
        print(" PInt = Proper Interval")
        
    def is_associative(self, verbose=False):
        result = True
        countskipped = 0
        countok = 0
        countfailed = 0
        counttotal = 0
        rels = self.identity
        for _a in rels:
            for _b in rels:
                for _c in rels:
                    if verbose:
                        print(f"{_a} x {_b} x {_c} :")
                    if (set(self.rel_range(_a)) & set(self.rel_domain(_b))) & (set(self.rel_range(_b)) & set(self.rel_domain(_c))):
                        a_rs = self.relset((_a,))
                        b_rs = self.relset((_b,))
                        c_rs = self.relset((_c,))
                        prod_ab = a_rs * b_rs
                        prod_bc = b_rs * c_rs
                        prod_ab_c = prod_ab * c_rs
                        prod_a_bc = a_rs * prod_bc
                        if not (prod_ab_c == prod_a_bc):
                            if verbose:
                                print(f"  Associativity fails for a = {a_rs}, b = {b_rs}, c = {c_rs}")
                                print(f"    associativity check: {self.rel_range(_a)}::{self.rel_domain(_b)} {self.rel_range(_b)}::{self.rel_domain(_c)}")
                                print(f"    (a * b) * c = {prod_ab_c}")
                                print(f"    a * (b * c) = {prod_a_bc}")
                            countfailed += 1
                            counttotal += 1
                            result = False
                        else:
                            if verbose:
                                print("  Associativity OK")
                            countok += 1
                            counttotal += 1
                    else:
                        if verbose:
                            print(f"  Skipping associativity check: {self.rel_range(_a)}::{self.rel_domain(_b)} {self.rel_range(_b)}::{self.rel_domain(_c)}")
                        countskipped += 1
                        counttotal += 1
        print(f"\nTEST SUMMARY: {countok} OK, {countskipped} Skipped, {countfailed} Failed ({counttotal} Total)")
        numrels = len(rels)
        totaltests = numrels * numrels * numrels
        if (counttotal != totaltests):
            print(f"Test counts do not add up; Total should be {totaltests}")
        return result

In [9]:
#alg = Algebra(os.path.join(path, "Algebras/IntervalAlgebra.json"))  # Allen's algebra of proper time intervals
#alg = Algebra(os.path.join(path, "Algebras/LeftBranchingIntervalAndPointAlgebra.json"))
alg = Algebra(os.path.join(path, "Algebras/RightBranchingIntervalAndPointAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/rcc8Algebra.json"))

print(alg)

<RightBranchingTimeIntervalAndPointAlgebra: Reich's right-branching extension to Allen's time interval algebra (see TIME-94 paper)>


In [10]:
alg.converse(alg.relset(('B','M','OI')))

RightBranchingTimeIntervalAndPointAlgebra(['BI', 'MI', 'O'])

In [11]:
alg.converse('B')

'BI'

In [12]:
alg.identity

RightBranchingTimeIntervalAndPointAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'PE', 'PF', 'PFI', 'PS', 'PSI', 'RB', 'RBI', 'RO', 'ROI', 'RS', 'R~', 'S', 'SI'])

In [13]:
alg.rel_name('B')

'Before'

In [14]:
#alg.transitivity_table

In [15]:
before = alg.relset(['B'])
during = alg.relset(['D'])

bxd = alg.mult(before, during)
bxd

RightBranchingTimeIntervalAndPointAlgebra(['B', 'D', 'M', 'O', 'PS', 'S'])

In [16]:
foobar = alg.relset(['D','M','F','SI'])
print(foobar.members())
alg.add(bxd,foobar)

('D', 'F', 'M', 'SI')


RightBranchingTimeIntervalAndPointAlgebra(['D', 'M'])

In [17]:
print(before.complement().members())
print(bxd.complement().members())

('BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'PE', 'PF', 'PFI', 'PS', 'PSI', 'RB', 'RBI', 'RO', 'ROI', 'RS', 'R~', 'S', 'SI')
('BI', 'DI', 'E', 'F', 'FI', 'MI', 'OI', 'PE', 'PF', 'PFI', 'PSI', 'RB', 'RBI', 'RO', 'ROI', 'RS', 'R~', 'SI')


In [18]:
alg.check_multiplication_identity()

True

See [Format String Syntax](https://docs.python.org/3/library/string.html#format-string-syntax)

In [19]:
alg.print_info()

  Algebra Name: RightBranchingTimeIntervalAndPointAlgebra
   Description: Reich's right-branching extension to Allen's time interval algebra (see TIME-94 paper)
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
             Before (  B)               After ( BI)    False      False       True    Pt|PInt       Pt|PInt
              After ( BI)              Before (  B)    False      False       True    Pt|PInt       Pt|PInt
             During (  D)            Contains ( DI)    False      False       True    Pt|PInt          PInt
           Contains ( DI)              During (  D)    False      False       True       PInt       Pt|PInt
             Equals (  E)              Equals (  E)     True       True       True       PInt          PInt
           Finishes (  F)         Finished-by ( FI)    False      False       True       PInt          PInt
        Finished-by ( FI)            Finishes (  F)    False      Fa

In [20]:
alg.is_associative()


TEST SUMMARY: 8560 OK, 5264 Skipped, 0 Failed (13824 Total)


True